# 통신사 고객 이탈(Churn) 예측을 위한 탐색적 데이터 분석(EDA)

본 노트북은 통신사 고객 이탈 데이터에 대한 종합적인 탐색적 데이터 분석을 수행합니다.

## 분석 목적
- 고객 이탈에 영향을 미치는 주요 요인 파악
- 데이터 품질 검증 및 전처리
- 고위험 고객 세그먼트 식별
- 머신러닝 모델링을 위한 인사이트 도출

## 데이터세트 (AWS Customer Churn 데이터세트)
- https://catalog.us-east-1.prod.workshops.aws/workshops/80ba0ea5-7cf9-4b8c-9d3f-1cd988b6c071/en-US/zzz-legacy/1-use-cases/1-marketing

## 바이브코딩 프로젝트
- 작성자: 동준상.넥스트플랫폼
- AI 도구: GPT, Cursor, Colab

## 1. 데이터 로드 및 기본 구조 파악

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

# 설정
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
sns.set_style('whitegrid')

# 한글 폰트 설정 (선택사항)
plt.rcParams['axes.unicode_minus'] = False

print("라이브러리 로드 완료")

In [ ]:
# 데이터 로드
df = pd.read_csv('churn.csv')

print("=" * 60)
print("데이터셋 기본 정보")
print("=" * 60)
print(f"\n데이터 Shape: {df.shape}")
print(f"총 행(Rows): {df.shape[0]:,}")
print(f"총 열(Columns): {df.shape[1]}")

# 데이터 미리보기
print("\n[처음 5개 행]")
display(df.head())

print("\n[마지막 5개 행]")
display(df.tail())

In [ ]:
# 데이터 타입 및 메모리 사용량 확인
print("=" * 60)
print("컬럼 정보 및 데이터 타입")
print("=" * 60)
df.info()

print("\n" + "=" * 60)
print("컬럼별 데이터 타입 요약")
print("=" * 60)
dtype_summary = df.dtypes.value_counts()
print(dtype_summary)

In [ ]:
# 결측치 확인
print("=" * 60)
print("결측치 현황")
print("=" * 60)

missing_stats = pd.DataFrame({
    '결측치 수': df.isnull().sum(),
    '결측치 비율(%)': (df.isnull().sum() / len(df) * 100).round(2)
})

missing_stats = missing_stats[missing_stats['결측치 수'] > 0].sort_values('결측치 수', ascending=False)

if len(missing_stats) > 0:
    print(missing_stats)
else:
    print("✓ 결측치가 없습니다!")

In [ ]:
# 타겟 변수(Churn) 분포 확인
print("=" * 60)
print("타겟 변수(Churn) 분포")
print("=" * 60)

churn_counts = df['Churn'].value_counts()
churn_pct = df['Churn'].value_counts(normalize=True) * 100

churn_summary = pd.DataFrame({
    '고객 수': churn_counts,
    '비율(%)': churn_pct.round(2)
})

print(churn_summary)

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 막대 그래프
churn_counts.plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
axes[0].set_title('고객 이탈 분포 (건수)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('이탈 여부')
axes[0].set_ylabel('고객 수')
axes[0].tick_params(axis='x', rotation=0)

for i, v in enumerate(churn_counts):
    axes[0].text(i, v + 50, str(v), ha='center', fontweight='bold')

# 파이 차트
colors = ['#2ecc71', '#e74c3c']
axes[1].pie(churn_counts, labels=churn_counts.index, autopct='%1.1f%%',
            colors=colors, startangle=90, textprops={'fontsize': 12, 'fontweight': 'bold'})
axes[1].set_title('고객 이탈 비율', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# 클래스 불균형 확인
imbalance_ratio = churn_counts.max() / churn_counts.min()
print(f"\n⚠️ 클래스 불균형 비율: {imbalance_ratio:.2f}:1")
if imbalance_ratio > 3:
    print("   → 클래스 불균형이 존재합니다. 모델링 시 SMOTE, 가중치 조정 등을 고려하세요.")
else:
    print("   → 클래스 분포가 비교적 균형적입니다.")

## 2. 데이터 전처리 및 품질 검증

In [ ]:
# 중복 데이터 확인
print("=" * 60)
print("중복 데이터 확인")
print("=" * 60)

duplicates = df.duplicated().sum()
print(f"중복된 행 수: {duplicates}")

if duplicates > 0:
    print("\n[중복 행 샘플]")
    display(df[df.duplicated(keep=False)].head())

    # 중복 제거
    df_clean = df.drop_duplicates()
    print(f"\n중복 제거 후 Shape: {df_clean.shape}")
else:
    print("✓ 중복 데이터가 없습니다.")
    df_clean = df.copy()

In [ ]:
# 데이터 타입 분류
print("=" * 60)
print("변수 타입 분류")
print("=" * 60)

# 수치형 변수와 범주형 변수 분리
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df_clean.select_dtypes(include=['object', 'bool']).columns.tolist()

# Phone 컬럼은 고유 식별자이므로 제외
if 'Phone' in categorical_cols:
    categorical_cols.remove('Phone')

print(f"\n수치형 변수 ({len(numeric_cols)}개):")
print(numeric_cols)

print(f"\n범주형 변수 ({len(categorical_cols)}개):")
print(categorical_cols)

In [ ]:
# 범주형 변수 유니크 값 확인
print("=" * 60)
print("범주형 변수 유니크 값 분석")
print("=" * 60)

for col in categorical_cols:
    unique_count = df_clean[col].nunique()
    print(f"\n[{col}]")
    print(f"  유니크 값 개수: {unique_count}")

    if unique_count <= 10:
        print(f"  값 분포:")
        print(df_clean[col].value_counts())
    else:
        print(f"  상위 10개 값:")
        print(df_clean[col].value_counts().head(10))

In [ ]:
# 이상치 탐지 - 박스플롯
print("=" * 60)
print("수치형 변수 이상치 탐지")
print("=" * 60)

# 박스플롯으로 이상치 시각화
fig, axes = plt.subplots(5, 3, figsize=(18, 20))
axes = axes.flatten()

for idx, col in enumerate(numeric_cols[:15]):
    sns.boxplot(data=df_clean, y=col, ax=axes[idx], color='skyblue')
    axes[idx].set_title(f'{col}', fontweight='bold')
    axes[idx].set_ylabel('')

# 사용하지 않는 서브플롯 제거
for idx in range(len(numeric_cols), len(axes)):
    fig.delaxes(axes[idx])

plt.suptitle('수치형 변수 박스플롯 (이상치 탐지)', fontsize=16, fontweight='bold', y=1.0)
plt.tight_layout()
plt.show()

In [ ]:
# IQR 방법으로 이상치 개수 확인
print("=" * 60)
print("IQR 방법 이상치 통계")
print("=" * 60)

outlier_summary = []

for col in numeric_cols:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df_clean[(df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)]
    outlier_count = len(outliers)
    outlier_pct = (outlier_count / len(df_clean)) * 100

    outlier_summary.append({
        '변수': col,
        '이상치 개수': outlier_count,
        '이상치 비율(%)': round(outlier_pct, 2),
        '하한': round(lower_bound, 2),
        '상한': round(upper_bound, 2)
    })

outlier_df = pd.DataFrame(outlier_summary)
outlier_df = outlier_df[outlier_df['이상치 개수'] > 0].sort_values('이상치 개수', ascending=False)

if len(outlier_df) > 0:
    display(outlier_df)
    print("\n⚠️ 이상치가 발견되었습니다. 도메인 지식을 바탕으로 처리 방법을 결정하세요.")
else:
    print("✓ IQR 기준 이상치가 없습니다.")

## 3. 단변량 분석 (Univariate Analysis)

In [ ]:
# 수치형 변수 기술 통계량
print("=" * 60)
print("수치형 변수 기술 통계량")
print("=" * 60)

desc_stats = df_clean[numeric_cols].describe().T
desc_stats['range'] = desc_stats['max'] - desc_stats['min']
desc_stats['cv'] = (desc_stats['std'] / desc_stats['mean']) * 100  # 변동계수

display(desc_stats.round(2))

In [ ]:
# 주요 수치형 변수 분포 시각화
print("=" * 60)
print("주요 수치형 변수 분포")
print("=" * 60)

# 주요 변수 선정
key_numeric_vars = ['Account_Length', 'Day_Mins', 'Eve_Mins', 'Night_Mins',
                    'Intl_Mins', 'CustServ_Calls', 'Day_Charge', 'Eve_Charge']

# 존재하는 변수만 필터링
key_numeric_vars = [col for col in key_numeric_vars if col in df_clean.columns]

fig, axes = plt.subplots(4, 2, figsize=(16, 16))
axes = axes.flatten()

for idx, col in enumerate(key_numeric_vars[:8]):
    # 히스토그램 + KDE
    df_clean[col].hist(bins=50, ax=axes[idx], alpha=0.7, color='skyblue', edgecolor='black')
    axes[idx].set_title(f'{col} 분포', fontweight='bold', fontsize=12)
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('빈도')

    # 평균선 추가
    mean_val = df_clean[col].mean()
    axes[idx].axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'평균: {mean_val:.2f}')
    axes[idx].legend()

plt.suptitle('주요 수치형 변수 분포 분석', fontsize=16, fontweight='bold', y=1.0)
plt.tight_layout()
plt.show()

In [ ]:
# 범주형 변수 빈도 분석
print("=" * 60)
print("범주형 변수 빈도 분석")
print("=" * 60)

# Churn 제외한 범주형 변수
cat_vars_to_plot = [col for col in categorical_cols if col != 'Churn']

fig, axes = plt.subplots(1, len(cat_vars_to_plot), figsize=(6*len(cat_vars_to_plot), 5))

if len(cat_vars_to_plot) == 1:
    axes = [axes]

for idx, col in enumerate(cat_vars_to_plot):
    value_counts = df_clean[col].value_counts()

    if len(value_counts) <= 20:  # 20개 이하인 경우만 시각화
        value_counts.plot(kind='bar', ax=axes[idx], color='coral')
        axes[idx].set_title(f'{col} 분포', fontweight='bold', fontsize=12)
        axes[idx].set_xlabel(col)
        axes[idx].set_ylabel('빈도')
        axes[idx].tick_params(axis='x', rotation=45)

        # 값 표시
        for i, v in enumerate(value_counts):
            axes[idx].text(i, v + value_counts.max()*0.01, str(v), ha='center', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# State(주) 분포 분석
if 'State' in df_clean.columns:
    print("=" * 60)
    print("주(State)별 고객 분포")
    print("=" * 60)

    state_counts = df_clean['State'].value_counts().head(20)

    plt.figure(figsize=(14, 6))
    state_counts.plot(kind='bar', color='steelblue')
    plt.title('상위 20개 주(State)별 고객 분포', fontweight='bold', fontsize=14)
    plt.xlabel('State')
    plt.ylabel('고객 수')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    print(f"\n총 주(State) 개수: {df_clean['State'].nunique()}")
    print(f"최다 고객 주: {state_counts.index[0]} ({state_counts.values[0]}명)")
    print(f"최소 고객 주: {df_clean['State'].value_counts().index[-1]} ({df_clean['State'].value_counts().values[-1]}명)")

## 4. 이변량 분석 및 고객 세그먼트별 패턴 탐색

In [ ]:
# Churn 여부에 따른 수치형 변수 비교
print("=" * 60)
print("Churn 여부에 따른 수치형 변수 비교")
print("=" * 60)

churn_comparison = df_clean.groupby('Churn')[numeric_cols].mean().T
churn_comparison['차이'] = churn_comparison[True] - churn_comparison[False]
churn_comparison['차이율(%)'] = ((churn_comparison[True] - churn_comparison[False]) / churn_comparison[False] * 100).round(2)

churn_comparison = churn_comparison.sort_values('차이', ascending=False)
display(churn_comparison.round(3))

In [ ]:
# 주요 변수에 대한 Churn 별 분포 비교
print("=" * 60)
print("Churn 여부에 따른 주요 변수 분포 비교")
print("=" * 60)

key_vars = ['Day_Mins', 'Eve_Mins', 'Night_Mins', 'Intl_Mins', 'CustServ_Calls', 'Account_Length']
key_vars = [col for col in key_vars if col in df_clean.columns]

fig, axes = plt.subplots(3, 2, figsize=(16, 14))
axes = axes.flatten()

for idx, col in enumerate(key_vars[:6]):
    # 박스플롯
    df_clean.boxplot(column=col, by='Churn', ax=axes[idx], patch_artist=True)
    axes[idx].set_title(f'{col} by Churn', fontweight='bold')
    axes[idx].set_xlabel('Churn')
    axes[idx].set_ylabel(col)
    plt.sca(axes[idx])
    plt.xticks([1, 2], ['False', 'True'])

plt.suptitle('Churn 여부에 따른 주요 변수 분포', fontsize=16, fontweight='bold', y=1.0)
plt.tight_layout()
plt.show()

In [ ]:
# T-검정: Churn 여부에 따른 수치형 변수 차이 유의성 검정
print("=" * 60)
print("T-검정: Churn 여부에 따른 변수 차이 유의성")
print("=" * 60)

ttest_results = []

for col in numeric_cols:
    churn_true = df_clean[df_clean['Churn'] == True][col].dropna()
    churn_false = df_clean[df_clean['Churn'] == False][col].dropna()

    if len(churn_true) > 0 and len(churn_false) > 0:
        t_stat, p_value = stats.ttest_ind(churn_true, churn_false)

        ttest_results.append({
            '변수': col,
            'T-통계량': round(t_stat, 4),
            'P-value': round(p_value, 6),
            '유의성': '***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'NS',
            'Churn=True 평균': round(churn_true.mean(), 3),
            'Churn=False 평균': round(churn_false.mean(), 3)
        })

ttest_df = pd.DataFrame(ttest_results).sort_values('P-value')
display(ttest_df)

print("\n유의성 기호: *** p<0.001, ** p<0.01, * p<0.05, NS=Not Significant")

In [ ]:
# 범주형 변수와 Churn 관계 분석
print("=" * 60)
print("범주형 변수와 Churn 관계 분석")
print("=" * 60)

cat_vars_analysis = [col for col in categorical_cols if col != 'Churn']

for col in cat_vars_analysis:
    print(f"\n[{col} vs Churn]")

    # 교차표
    crosstab = pd.crosstab(df_clean[col], df_clean['Churn'], margins=True)
    print(crosstab)

    # 카이제곱 검정
    if df_clean[col].nunique() <= 50:  # 범주가 너무 많으면 제외
        contingency_table = pd.crosstab(df_clean[col], df_clean['Churn'])
        chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)

        print(f"\n카이제곱 통계량: {chi2:.4f}")
        print(f"P-value: {p_value:.6f}")
        print(f"자유도: {dof}")

        if p_value < 0.05:
            print("✓ 유의한 관계가 있습니다 (p < 0.05)")
        else:
            print("✗ 유의한 관계가 없습니다 (p >= 0.05)")

In [ ]:
# 국제 플랜/음성메일 플랜과 이탈률
if 'Intl_Plan' in df_clean.columns and 'Vmail_Plan' in df_clean.columns:
    print("=" * 60)
    print("플랜 가입 여부와 이탈률")
    print("=" * 60)

    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # 국제 플랜
    intl_churn = pd.crosstab(df_clean['Intl_Plan'], df_clean['Churn'], normalize='index') * 100
    intl_churn.plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'], stacked=False)
    axes[0].set_title('국제 플랜별 이탈률', fontweight='bold', fontsize=14)
    axes[0].set_xlabel('International Plan')
    axes[0].set_ylabel('비율 (%)')
    axes[0].legend(['유지', '이탈'])
    axes[0].tick_params(axis='x', rotation=0)

    # 음성메일 플랜
    vmail_churn = pd.crosstab(df_clean['Vmail_Plan'], df_clean['Churn'], normalize='index') * 100
    vmail_churn.plot(kind='bar', ax=axes[1], color=['#2ecc71', '#e74c3c'], stacked=False)
    axes[1].set_title('음성메일 플랜별 이탈률', fontweight='bold', fontsize=14)
    axes[1].set_xlabel('Voicemail Plan')
    axes[1].set_ylabel('비율 (%)')
    axes[1].legend(['유지', '이탈'])
    axes[1].tick_params(axis='x', rotation=0)

    plt.tight_layout()
    plt.show()

In [ ]:
# 고객 서비스 통화 횟수와 이탈
if 'CustServ_Calls' in df_clean.columns:
    print("=" * 60)
    print("고객 서비스 통화 횟수와 이탈 관계")
    print("=" * 60)

    # 통화 횟수별 이탈률
    custserv_churn = df_clean.groupby('CustServ_Calls')['Churn'].agg(['sum', 'count'])
    custserv_churn['churn_rate'] = (custserv_churn['sum'] / custserv_churn['count'] * 100).round(2)
    custserv_churn.columns = ['이탈 고객 수', '총 고객 수', '이탈률(%)']

    print(custserv_churn)

    # 시각화
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # 이탈률 선 그래프
    axes[0].plot(custserv_churn.index, custserv_churn['이탈률(%)'], marker='o', linewidth=2, markersize=8, color='#e74c3c')
    axes[0].set_title('고객 서비스 통화 횟수별 이탈률', fontweight='bold', fontsize=14)
    axes[0].set_xlabel('고객 서비스 통화 횟수')
    axes[0].set_ylabel('이탈률 (%)')
    axes[0].grid(True, alpha=0.3)

    # 고객 수 분포
    custserv_churn['총 고객 수'].plot(kind='bar', ax=axes[1], color='steelblue')
    axes[1].set_title('고객 서비스 통화 횟수별 고객 분포', fontweight='bold', fontsize=14)
    axes[1].set_xlabel('고객 서비스 통화 횟수')
    axes[1].set_ylabel('고객 수')

    plt.tight_layout()
    plt.show()

    print(f"\n⚠️ 인사이트: 고객 서비스 통화 횟수가 증가할수록 이탈률이 {'증가' if custserv_churn['이탈률(%)'].iloc[-1] > custserv_churn['이탈률(%)'].iloc[0] else '감소'}하는 경향")

In [ ]:
# 지역(State)별 이탈률 분석
if 'State' in df_clean.columns:
    print("=" * 60)
    print("지역(State)별 이탈률 분석")
    print("=" * 60)

    state_churn = df_clean.groupby('State')['Churn'].agg(['sum', 'count'])
    state_churn['churn_rate'] = (state_churn['sum'] / state_churn['count'] * 100).round(2)
    state_churn = state_churn.sort_values('churn_rate', ascending=False)

    # 상위/하위 10개 주
    top10_states = state_churn.head(10)
    bottom10_states = state_churn.tail(10)

    print("\n[이탈률 상위 10개 주]")
    print(top10_states)

    print("\n[이탈률 하위 10개 주]")
    print(bottom10_states)

    # 시각화
    fig, axes = plt.subplots(2, 1, figsize=(16, 10))

    # 상위 10개
    top10_states['churn_rate'].plot(kind='barh', ax=axes[0], color='#e74c3c')
    axes[0].set_title('이탈률 상위 10개 주', fontweight='bold', fontsize=14)
    axes[0].set_xlabel('이탈률 (%)')
    axes[0].invert_yaxis()

    # 하위 10개
    bottom10_states['churn_rate'].plot(kind='barh', ax=axes[1], color='#2ecc71')
    axes[1].set_title('이탈률 하위 10개 주', fontweight='bold', fontsize=14)
    axes[1].set_xlabel('이탈률 (%)')
    axes[1].invert_yaxis()

    plt.tight_layout()
    plt.show()

## 5. 다변량 분석 및 상관관계 분석

In [ ]:
# 상관관계 매트릭스
print("=" * 60)
print("수치형 변수 상관관계 분석")
print("=" * 60)

# Churn을 수치형으로 변환
df_corr = df_clean.copy()
df_corr['Churn_numeric'] = df_corr['Churn'].astype(int)

# 상관관계 계산
correlation_matrix = df_corr[numeric_cols + ['Churn_numeric']].corr()

# 히트맵
plt.figure(figsize=(16, 14))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='RdYlBu_r',
            center=0, square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('변수 간 상관관계 히트맵', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Churn과의 상관관계 (절대값 기준 정렬)
churn_corr = correlation_matrix['Churn_numeric'].drop('Churn_numeric').abs().sort_values(ascending=False)
print("\n[Churn과의 상관관계 (절대값 기준)]")
print(churn_corr)

In [ ]:
# 높은 상관관계 변수 쌍 찾기 (다중공선성 확인)
print("=" * 60)
print("높은 상관관계 변수 쌍 (다중공선성 확인)")
print("=" * 60)

# 상관계수 절대값 0.8 이상인 쌍 찾기
high_corr_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) >= 0.8:
            high_corr_pairs.append({
                '변수1': correlation_matrix.columns[i],
                '변수2': correlation_matrix.columns[j],
                '상관계수': round(correlation_matrix.iloc[i, j], 4)
            })

if len(high_corr_pairs) > 0:
    high_corr_df = pd.DataFrame(high_corr_pairs).sort_values('상관계수', key=abs, ascending=False)
    display(high_corr_df)
    print("\n⚠️ 다중공선성 위험: 상관계수가 높은 변수들이 발견되었습니다.")
    print("   모델링 시 변수 선택(VIF 확인) 또는 차원 축소를 고려하세요.")
else:
    print("✓ 상관계수 0.8 이상인 변수 쌍이 없습니다.")

In [ ]:
# 통화 시간(Mins)과 요금(Charge) 변수 관계 분석
print("=" * 60)
print("통화 시간과 요금 변수 관계")
print("=" * 60)

time_charge_pairs = [
    ('Day_Mins', 'Day_Charge'),
    ('Eve_Mins', 'Eve_Charge'),
    ('Night_Mins', 'Night_Charge'),
    ('Intl_Mins', 'Intl_Charge')
]

# 존재하는 변수 쌍만 필터링
existing_pairs = [(m, c) for m, c in time_charge_pairs if m in df_clean.columns and c in df_clean.columns]

if len(existing_pairs) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()

    for idx, (mins_col, charge_col) in enumerate(existing_pairs[:4]):
        axes[idx].scatter(df_clean[mins_col], df_clean[charge_col], alpha=0.5, s=10)
        axes[idx].set_title(f'{mins_col} vs {charge_col}', fontweight='bold')
        axes[idx].set_xlabel(mins_col)
        axes[idx].set_ylabel(charge_col)

        # 상관계수 표시
        corr = df_clean[[mins_col, charge_col]].corr().iloc[0, 1]
        axes[idx].text(0.05, 0.95, f'상관계수: {corr:.4f}',
                      transform=axes[idx].transAxes, fontsize=11,
                      verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    plt.suptitle('통화 시간과 요금의 관계', fontsize=16, fontweight='bold', y=1.0)
    plt.tight_layout()
    plt.show()

    print("\n💡 통화 시간과 요금은 완벽한 선형 관계를 보입니다.")
    print("   모델링 시 둘 중 하나만 사용하는 것을 권장합니다.")

In [ ]:
# 주요 변수 페어플롯
print("=" * 60)
print("주요 변수 페어플롯 (Churn 기준)")
print("=" * 60)

# 주요 변수 선택 (너무 많으면 시각화가 복잡함)
pairplot_vars = ['Day_Mins', 'Eve_Mins', 'Night_Mins', 'CustServ_Calls', 'Churn']
pairplot_vars = [col for col in pairplot_vars if col in df_clean.columns]

if len(pairplot_vars) > 2:
    # 샘플링 (데이터가 많으면 시간이 오래 걸림)
    sample_size = min(1000, len(df_clean))
    df_sample = df_clean[pairplot_vars].sample(n=sample_size, random_state=42)

    pairplot = sns.pairplot(df_sample, hue='Churn', palette={True: '#e74c3c', False: '#2ecc71'},
                           diag_kind='kde', plot_kws={'alpha': 0.6, 's': 20}, height=2.5)
    pairplot.fig.suptitle(f'주요 변수 페어플롯 (샘플 {sample_size}개)', fontsize=16, fontweight='bold', y=1.0)
    plt.tight_layout()
    plt.show()

In [ ]:
# PCA 차원 축소 가능성 검토
print("=" * 60)
print("PCA 차원 축소 분석")
print("=" * 60)

# 수치형 변수만 사용
from sklearn.preprocessing import StandardScaler

# 데이터 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clean[numeric_cols].fillna(0))

# PCA 적용
pca = PCA()
pca.fit(X_scaled)

# 설명된 분산 비율
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 주성분별 설명 분산
axes[0].bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.7, color='steelblue')
axes[0].set_title('주성분별 설명된 분산 비율', fontweight='bold', fontsize=14)
axes[0].set_xlabel('주성분')
axes[0].set_ylabel('설명된 분산 비율')
axes[0].grid(True, alpha=0.3)

# 누적 설명 분산
axes[1].plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linewidth=2, markersize=8, color='#e74c3c')
axes[1].axhline(y=0.95, color='green', linestyle='--', linewidth=2, label='95% 분산')
axes[1].axhline(y=0.90, color='orange', linestyle='--', linewidth=2, label='90% 분산')
axes[1].set_title('누적 설명된 분산 비율', fontweight='bold', fontsize=14)
axes[1].set_xlabel('주성분 개수')
axes[1].set_ylabel('누적 설명된 분산 비율')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 95% 분산 설명에 필요한 주성분 개수
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
n_components_90 = np.argmax(cumulative_variance >= 0.90) + 1

print(f"\n90% 분산 설명에 필요한 주성분 개수: {n_components_90} / {len(numeric_cols)}")
print(f"95% 분산 설명에 필요한 주성분 개수: {n_components_95} / {len(numeric_cols)}")

if n_components_95 < len(numeric_cols) * 0.7:
    print("\n💡 PCA를 통한 차원 축소가 효과적일 수 있습니다.")
else:
    print("\n💡 대부분의 주성분이 필요하므로 차원 축소 효과가 제한적입니다.")

## 6. 핵심 인사이트 도출 및 비즈니스 권장사항

In [ ]:
# 이탈 고객의 주요 특성 요약
print("=" * 80)
print("이탈 고객(Churn=True)의 주요 특성")
print("=" * 80)

churn_true = df_clean[df_clean['Churn'] == True]
churn_false = df_clean[df_clean['Churn'] == False]

print(f"\n총 고객 수: {len(df_clean):,}")
print(f"이탈 고객 수: {len(churn_true):,} ({len(churn_true)/len(df_clean)*100:.2f}%)")
print(f"유지 고객 수: {len(churn_false):,} ({len(churn_false)/len(df_clean)*100:.2f}%)")

print("\n" + "=" * 80)
print("이탈 고객 vs 유지 고객 주요 지표 비교")
print("=" * 80)

key_metrics = ['Day_Mins', 'Eve_Mins', 'Night_Mins', 'Intl_Mins', 'CustServ_Calls', 'Account_Length']
key_metrics = [col for col in key_metrics if col in df_clean.columns]

comparison_summary = []

for metric in key_metrics:
    churn_mean = churn_true[metric].mean()
    no_churn_mean = churn_false[metric].mean()
    diff = churn_mean - no_churn_mean
    diff_pct = (diff / no_churn_mean * 100) if no_churn_mean != 0 else 0

    comparison_summary.append({
        '지표': metric,
        '이탈 고객 평균': round(churn_mean, 2),
        '유지 고객 평균': round(no_churn_mean, 2),
        '차이': round(diff, 2),
        '차이율(%)': round(diff_pct, 2)
    })

comparison_df = pd.DataFrame(comparison_summary).sort_values('차이율(%)', key=abs, ascending=False)
display(comparison_df)

In [ ]:
# 주요 위험 신호(Risk Indicators) 식별
print("=" * 80)
print("이탈 위험 신호 (Risk Indicators)")
print("=" * 80)

risk_indicators = []

# 1. 고객 서비스 통화 횟수
if 'CustServ_Calls' in df_clean.columns:
    high_calls_churn_rate = df_clean[df_clean['CustServ_Calls'] >= 4]['Churn'].mean() * 100
    low_calls_churn_rate = df_clean[df_clean['CustServ_Calls'] < 4]['Churn'].mean() * 100

    risk_indicators.append({
        '위험 신호': '고객 서비스 통화 4회 이상',
        '이탈률(%)': round(high_calls_churn_rate, 2),
        '기준 이탈률(%)': round(low_calls_churn_rate, 2),
        '위험도': f"{high_calls_churn_rate/low_calls_churn_rate:.2f}배"
    })

# 2. 국제 플랜
if 'Intl_Plan' in df_clean.columns:
    intl_yes_churn = df_clean[df_clean['Intl_Plan'] == 'yes']['Churn'].mean() * 100
    intl_no_churn = df_clean[df_clean['Intl_Plan'] == 'no']['Churn'].mean() * 100

    if intl_yes_churn > intl_no_churn:
        risk_indicators.append({
            '위험 신호': '국제 플랜 가입',
            '이탈률(%)': round(intl_yes_churn, 2),
            '기준 이탈률(%)': round(intl_no_churn, 2),
            '위험도': f"{intl_yes_churn/intl_no_churn:.2f}배"
        })

# 3. 음성메일 플랜
if 'Vmail_Plan' in df_clean.columns:
    vmail_no_churn = df_clean[df_clean['Vmail_Plan'] == 'no']['Churn'].mean() * 100
    vmail_yes_churn = df_clean[df_clean['Vmail_Plan'] == 'yes']['Churn'].mean() * 100

    if vmail_no_churn > vmail_yes_churn:
        risk_indicators.append({
            '위험 신호': '음성메일 플랜 미가입',
            '이탈률(%)': round(vmail_no_churn, 2),
            '기준 이탈률(%)': round(vmail_yes_churn, 2),
            '위험도': f"{vmail_no_churn/vmail_yes_churn:.2f}배"
        })

# 4. 주간 통화 시간
if 'Day_Mins' in df_clean.columns:
    day_mins_75 = df_clean['Day_Mins'].quantile(0.75)
    high_day_mins_churn = df_clean[df_clean['Day_Mins'] >= day_mins_75]['Churn'].mean() * 100
    low_day_mins_churn = df_clean[df_clean['Day_Mins'] < day_mins_75]['Churn'].mean() * 100

    if high_day_mins_churn > low_day_mins_churn * 1.2:
        risk_indicators.append({
            '위험 신호': f'주간 통화시간 상위 25% ({day_mins_75:.1f}분 이상)',
            '이탈률(%)': round(high_day_mins_churn, 2),
            '기준 이탈률(%)': round(low_day_mins_churn, 2),
            '위험도': f"{high_day_mins_churn/low_day_mins_churn:.2f}배"
        })

if len(risk_indicators) > 0:
    risk_df = pd.DataFrame(risk_indicators).sort_values('이탈률(%)', ascending=False)
    display(risk_df)
else:
    print("위험 신호를 식별할 수 없습니다.")

In [ ]:
# 고위험 고객 세그먼트 정의
print("=" * 80)
print("고위험 고객 세그먼트 식별")
print("=" * 80)

# 고위험 조건 정의
high_risk_conditions = []

if 'CustServ_Calls' in df_clean.columns:
    high_risk_conditions.append(df_clean['CustServ_Calls'] >= 4)

if 'Intl_Plan' in df_clean.columns:
    high_risk_conditions.append(df_clean['Intl_Plan'] == 'yes')

if 'Vmail_Plan' in df_clean.columns:
    high_risk_conditions.append(df_clean['Vmail_Plan'] == 'no')

# 여러 위험 요소를 가진 고객
if len(high_risk_conditions) >= 2:
    risk_count = sum(high_risk_conditions)

    for i in range(len(high_risk_conditions), 0, -1):
        segment = df_clean[risk_count >= i]
        if len(segment) > 0:
            churn_rate = segment['Churn'].mean() * 100
            print(f"\n위험 요소 {i}개 이상 보유 고객:")
            print(f"  - 고객 수: {len(segment):,} ({len(segment)/len(df_clean)*100:.2f}%)")
            print(f"  - 이탈률: {churn_rate:.2f}%")
            print(f"  - 실제 이탈 고객 수: {segment['Churn'].sum():,}")

# 고위험 고객 예시
if len(high_risk_conditions) >= 2:
    very_high_risk = df_clean[risk_count >= len(high_risk_conditions)]

    if len(very_high_risk) > 0:
        print(f"\n\n⚠️ 최고위험 고객 ({len(high_risk_conditions)}개 위험 요소 모두 보유):")
        print(f"   총 {len(very_high_risk):,}명, 이탈률 {very_high_risk['Churn'].mean()*100:.2f}%")
        print("\n[샘플 5명]")
        display(very_high_risk.head())

In [ ]:
# 머신러닝 모델링을 위한 주요 피처 추천
print("=" * 80)
print("머신러닝 모델링을 위한 주요 피처 추천")
print("=" * 80)

# 1. Churn과 상관관계가 높은 변수
if 'Churn_numeric' in df_corr.columns:
    top_corr_features = correlation_matrix['Churn_numeric'].drop('Churn_numeric').abs().sort_values(ascending=False).head(10)

    print("\n[1] Churn과 상관관계 상위 10개 변수:")
    for feature, corr in top_corr_features.items():
        print(f"   {feature}: {corr:.4f}")

# 2. T-검정에서 유의한 변수
if len(ttest_df) > 0:
    significant_features = ttest_df[ttest_df['유의성'] != 'NS']['변수'].tolist()
    print(f"\n[2] T-검정 유의한 변수 ({len(significant_features)}개):")
    print(f"   {', '.join(significant_features[:15])}")
    if len(significant_features) > 15:
        print(f"   ... 외 {len(significant_features) - 15}개")

# 3. 다중공선성 제거 권장사항
print("\n[3] 변수 선택 권장사항:")
print("   - 통화시간(Mins)과 요금(Charge) 변수는 완벽한 상관관계 → 둘 중 하나만 사용")
print("     → 권장: Mins 변수 사용 (해석이 직관적)")
print("   - Phone 변수는 고유 식별자 → 제거")
print("   - Area_Code는 예측력이 낮으면 제거 고려")

# 4. 범주형 변수 인코딩
print("\n[4] 범주형 변수 인코딩 방법:")
print("   - State: One-Hot Encoding 또는 Target Encoding")
print("   - Intl_Plan, Vmail_Plan: Binary Encoding (yes=1, no=0)")

# 5. 최종 추천 피처 세트
recommended_features = [
    'CustServ_Calls',      # 가장 강력한 예측 변수
    'Intl_Plan',           # 이탈률 차이가 큼
    'Vmail_Plan',          # 보호 효과
    'Day_Mins',            # 주간 사용 패턴
    'Eve_Mins',            # 저녁 사용 패턴
    'Night_Mins',          # 야간 사용 패턴
    'Intl_Mins',           # 국제 통화 패턴
    'Vmail_Message',       # 음성메일 사용
    'Account_Length',      # 계정 기간
    'Day_Calls',           # 통화 빈도
    'State'                # 지역별 차이
]

# 존재하는 피처만 필터링
recommended_features = [f for f in recommended_features if f in df_clean.columns]

print(f"\n[5] 최종 추천 피처 세트 ({len(recommended_features)}개):")
for i, feature in enumerate(recommended_features, 1):
    print(f"   {i}. {feature}")

In [ ]:
# 데이터 기반 마케팅 전략 제안
print("=" * 80)
print("데이터 기반 이탈 방지 마케팅 전략")
print("=" * 80)

strategies = [
    {
        '전략': '고객 서비스 품질 개선',
        '근거': f'고객 서비스 통화 4회 이상 고객의 이탈률이 매우 높음',
        '실행방안': [
            '- 3회 이상 통화 고객에 대한 선제적 케어',
            '- 고객 서비스 팀 교육 및 프로세스 개선',
            '- FAQ/셀프서비스 포털 강화로 통화 빈도 감소'
        ]
    },
    {
        '전략': '국제 플랜 고객 특별 관리',
        '근거': '국제 플랜 가입 고객의 이탈률이 높음',
        '실행방안': [
            '- 국제 플랜 요금제 경쟁력 검토',
            '- 국제 플랜 가입 고객 대상 로열티 프로그램',
            '- 국제 통화 품질 개선 및 모니터링'
        ]
    },
    {
        '전략': '음성메일 플랜 가입 촉진',
        '근거': '음성메일 플랜 미가입 고객의 이탈률이 높음',
        '실행방안': [
            '- 음성메일 플랜 무료 체험 기간 제공',
            '- 음성메일 플랜의 가치 제안 강화',
            '- 번들 상품 개발 (음성메일 + 기타 서비스)'
        ]
    },
    {
        '전략': '고사용량 고객 리텐션 프로그램',
        '근거': '통화 시간이 많은 고객군의 이탈률 증가 경향',
        '실행방안': [
            '- 고사용량 고객 대상 맞춤형 요금제 제안',
            '- 무제한 플랜 또는 할인 혜택 제공',
            '- VIP 고객 프로그램 운영'
        ]
    },
    {
        '전략': '지역별 맞춤 전략',
        '근거': '주(State)별로 이탈률 차이가 존재',
        '실행방안': [
            '- 고이탈률 지역 원인 분석 (네트워크 품질, 경쟁사 등)',
            '- 지역별 맞춤 프로모션 및 캠페인',
            '- 지역 네트워크 품질 개선 투자'
        ]
    },
    {
        '전략': '예측 모델 기반 선제적 대응',
        '근거': 'EDA 결과를 바탕으로 한 이탈 예측 모델 구축 가능',
        '실행방안': [
            '- 머신러닝 이탈 예측 모델 개발 및 배포',
            '- 고위험 고객 조기 식별 시스템 구축',
            '- 자동화된 리텐션 캠페인 트리거'
        ]
    }
]

for i, strategy in enumerate(strategies, 1):
    print(f"\n[전략 {i}] {strategy['전략']}")
    print(f"근거: {strategy['근거']}")
    print("실행방안:")
    for action in strategy['실행방안']:
        print(f"  {action}")

print("\n" + "=" * 80)
print("우선순위: 고객 서비스 품질 개선 > 국제 플랜 관리 > 예측 모델 구축")
print("=" * 80)

In [ ]:
# 최종 요약 및 다음 단계
print("=" * 80)
print("EDA 최종 요약 및 다음 단계")
print("=" * 80)

print("\n📊 데이터 개요:")
print(f"   - 총 고객 수: {len(df_clean):,}")
print(f"   - 변수 수: {df_clean.shape[1]}")
print(f"   - 이탈률: {df_clean['Churn'].mean()*100:.2f}%")

print("\n🔍 주요 발견사항:")
print("   1. 고객 서비스 통화 횟수가 이탈의 가장 강력한 예측 변수")
print("   2. 국제 플랜 가입 고객의 이탈률이 높음")
print("   3. 음성메일 플랜은 이탈 방지 효과")
print("   4. 통화 시간과 요금은 완벽한 상관관계 (변수 선택 필요)")
print("   5. 지역별 이탈률 차이 존재")

print("\n📈 다음 단계:")
print("   1. 피처 엔지니어링")
print("      - 통화 패턴 파생 변수 생성 (시간대별 사용 비율 등)")
print("      - 고객 세그먼트 변수 생성")
print("      - 교호작용(Interaction) 변수 검토")
print("\n   2. 모델 개발")
print("      - Logistic Regression (베이스라인)")
print("      - Random Forest / XGBoost (앙상블)")
print("      - 클래스 불균형 처리 (SMOTE, 가중치 등)")
print("\n   3. 모델 평가")
print("      - Precision, Recall, F1-Score, AUC-ROC")
print("      - 비즈니스 메트릭 정의 (이탈 예방 비용 vs 혜택)")
print("\n   4. 배포 및 모니터링")
print("      - 실시간 이탈 예측 시스템 구축")
print("      - A/B 테스트를 통한 리텐션 전략 검증")
print("      - 모델 성능 모니터링 및 재학습 파이프라인")

print("\n" + "=" * 80)
print("✅ 탐색적 데이터 분석(EDA) 완료")
print("=" * 80)

In [ ]:
# 분석 결과 저장 (선택사항)
print("=" * 80)
print("분석 결과 저장")
print("=" * 80)

# 1. 전처리된 데이터 저장
# df_clean.to_csv('churn_cleaned.csv', index=False)
# print("✓ 전처리된 데이터 저장: churn_cleaned.csv")

# 2. 주요 통계 저장
# churn_comparison.to_csv('churn_comparison_stats.csv')
# print("✓ Churn 비교 통계 저장: churn_comparison_stats.csv")

# 3. 상관관계 매트릭스 저장
# correlation_matrix.to_csv('correlation_matrix.csv')
# print("✓ 상관관계 매트릭스 저장: correlation_matrix.csv")

print("\n💡 필요시 위 코드의 주석을 해제하여 결과를 저장하세요.")